Comparing the Random Graphs with the Respiratory Graphs 

Cell 1: Imports & Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

# Set aesthetic parameters for "publication-quality" plots
sns.set_theme(style="whitegrid", context="notebook", font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['lines.linewidth'] = 2.5

Cell 2: Load and Combine Data
This cell handles the two separate CSVs and prepares them for a unified analysis.

In [ ]:
# Import analysis utilities
from analysis_utils import setup_analysis_environment, load_all_data, load_experiment_data

# Setup environment and load all data
setup_analysis_environment()
# data = load_all_data()

# Extract data for use in notebook
df_random = load_experiment_data('random_graphs_grand_experiments.csv')
df_resp = load_experiment_data('comparative_study_results.csv')    # Respiratory graph experiments
df_graphs = load_experiment_data('graph_database.csv')       # Graph database

# df_experiments = pd.concat([df_random, df_resp])
df_experiments = df_random
if not df_experiments.empty:
    print(f"Total Combined Records: {len(df_experiments)}")
else:
    print("No experiment data loaded - check that data files exist in ../simulation_data/")

print("Shape before merging: ", df_experiments.shape)
df_experiments = pd.merge(
    df_experiments, 
    df_graphs, 
    on='wl_hash', 
    how='left', 
    suffixes=('', '_db')
)
df_experiments.shape


Cell 3: Aggregate Statistics
We need to calculate the probability of fixation and median times for every unique graph configuration.

In [ ]:
# Filter for successful fixations for time analysis
success_df = df_experiments[df_experiments['fixation'] == True]

# 1. Calculate P(fix) for every unique graph and r value
# Group by category, name, hash, and r
stats_prob = df_experiments.groupby(['wl_hash', 'r', 'graph_name',])['fixation'].mean().reset_index(name='prob_fixation')

# 2. Calculate Median Steps (Conditional on Fixation)
stats_time = success_df.groupby(['wl_hash', 'r', 'graph_name',])['steps'].median().reset_index(name='median_steps')

# 3. Merge metrics into a single analysis dataframe
analysis_df = pd.merge(stats_prob, stats_time, on=['wl_hash', 'r', 'graph_name',], how='outer')
analysis_df = pd.merge(
    analysis_df, 
    df_graphs, 
    on='wl_hash', 
    how='left', 
    suffixes=('', '_db')
)
analysis_df['z_order'] = np.where(analysis_df['category'] == 'Random', 0, 1)
analysis_df = analysis_df.sort_values('z_order')

analysis_df = analysis_df[analysis_df['category'] != 'Complete']

# Display sample
analysis_df.tail(20)

In [ ]:
analysis_df[analysis_df['n_edges'] >= 32].shape

Cell 4: Plot 1 - The "Trade-off" Map (Scatter)
This visualizes how different graphs are having different fixation time and fixation probability (also we can see if the correlation between the two vlaues is limited)

In [ ]:
from population_graph import GRAPH_PROPS
def plot_property_effect(df, x_prop, y_outcome='prob_fixation', color_dict=COLOR_DICT):
    """
    Plots a specific graph property against an evolutionary outcome.
    Faceted by 'r' to show how the effect varies with selection strength.
    """
    plt.figure(figsize=(11,8))
    is_prob = (y_outcome == 'prob_fixation')
    ylabel = "Probability of Fixation ($P_{fix}$)" if is_prob else "Median Steps (Time)"
    sns.scatterplot(
        data=df,
        x=x_prop,
        y=y_outcome,
        hue='category',     # Color by Category
        style='r',          # Shape by r
        palette=color_dict,
        s=120,              # Marker size
        alpha=0.85,         # Transparency
        edgecolor='w',      # White edge to make points pop
        linewidth=0.5
    )
    # Add Neutral Limit Line if plotting Probability
    if is_prob:
        avg_n = df['n_nodes'].mean()
        plt.axhline(1/avg_n, color='black', linestyle=':', label=f'Neutral (1/N)')

    plt.title(f'Effect of {x_prop.replace('_', ' ').title()} on {ylabel}', fontsize=16)
    plt.xlabel(x_prop.replace('_', ' ').title())
    plt.ylabel(ylabel)
    plt.grid(True, linestyle='--', alpha=0.4)
    
    # Legend handling: Place outside
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)
    
    plt.tight_layout()
    plt.show()

print(len(GRAPH_PROPS))
plot_property_effect(analysis_df, 'median_steps', 'prob_fixation')
# --- EXAMPLES OF USAGE ---
for prop in GRAPH_PROPS:
    plot_property_effect(analysis_df, prop, 'median_steps')

In [ ]:
not_in = []
for col in df.columns: 
    if col not in GRAPH_PROPS: not_in.append(col) 
print(not_in)